# Advanced

This tutorial will discuss some more advanced details of `ebcc`.

In [1]:
import numpy as np
from pyscf import gto, scf

mol = gto.M(
    atom="N 0 0 0; N 0 0 1",
    basis="aug-cc-pvdz",
    verbose=0,
)

mf = scf.RHF(mol)
mf.kernel()

-108.93589222112035

In [2]:
import sys
from logging import StreamHandler
from ebcc.core.logging import Logger

log = Logger("main")
log.setLevel(20)
log.addHandler(StreamHandler(sys.stdout))

Most classes (damping, Hamiltonian elements, etc.) that the CC solver uses are stored as attributes of the CC solver, and can be customised either by assignment for those used post-initialisation time, or by subclasses for those used during initialisation.

For example, the default damping class is `ebcc.core.damping.DIIS`, which includes a linear damping factor to allow both DIIS and linear damping. This can be replaced with either `ebcc.core.LinearDamping` for purely linear damping, or `ebcc.core.NoDamping` for none. Additionally, one can subclass `ebcc.core.BaseDamping` to define a custom damping procedure.

In [3]:
from ebcc import EBCC
from ebcc.core.damping import LinearDamping, NoDamping

ccsd = EBCC(mf, damping=0.5, log=log)
ccsd.Damping = LinearDamping
ccsd.kernel()

ccsd = EBCC(mf, log=log)
ccsd.Damping = NoDamping
ccsd.kernel()

        _
       | |
   ___ | |__    ___   ___
  / _ \| '_ \  / __| / __|
 |  __/| |_) || (__ | (__
  \___||_.__/  \___| \___|
                     1.5.0
numpy:
 > Version:  1.26.1
 > Git hash: N/A
pyscf:
 > Version:  2.6.2
 > Git hash: N/A
ebcc:
 > Version:  1.5.0
 > Git hash: N/A
OMP_NUM_THREADS = 1

RCCSD
Options:
 > e_tol:  1e-08
 > t_tol:  1e-08
 > max_iter:  200
 > diis_space:  9
 > diis_min_space:  1
 > damping:  0.5
Ansatz: CCSD
Space: (7o, 39v)
Solving for excitation amplitudes.
Iter   Energy (corr.)      Energy (tot.)     Δ(Energy)      Δ(Ampl.)
   0    -0.2946967101    -109.2305889312
   1    -0.2929471958    -109.2288394169     1.750e-03     1.223e-02
   2    -0.2970752405    -109.2329674616     4.128e-03     1.950e-03
   3    -0.3013688523    -109.2372610735     4.294e-03     2.265e-03
   4    -0.3017833195    -109.2376755406     4.145e-04     1.039e-03
   5    -0.3022113880    -109.2381036092     4.281e-04     7.884e-04
   6    -0.3025132218    -109.2384054430     3.018e-

-0.30291732446993286

The Hamiltonian elements are controlled by just-in-time containers that transform and cache blocks of integrals when they are requested. This allows the same integrals classes to be used for all ansatzes, without the need to transform redundant blocks of the MO basis integrals.

Warning that the following example may not behave as intended unless the notebook is run once with cells in a consecutive order, due to the global behaviour of symbols in a notebook.

In [4]:
from pyscf import lib
from ebcc import NullLogger

ccsd = EBCC(mf, log=NullLogger())
eris = ccsd.get_eris()

print(f"Memory usage (no integrals): {lib.current_memory()[0]:.3f} MB")

ovov = eris.ovov
print(f"(ov|ov) shape:", ovov.shape)

print(f"Memory usage (with (ov|ov)): {lib.current_memory()[0]:.3f} MB")

ovvv = eris.ovvv
print(f"(ov|vv) shape:", ovvv.shape)
vvvv = eris.vvvv
print(f"(vv|vv) shape:", vvvv.shape)

print(f"Memory usage (with (ov|ov), (ov|vv), (vv|vv)): {lib.current_memory()[0]:.3f} MB")

Memory usage (no integrals): 153.887 MB
(ov|ov) shape: (7, 39, 7, 39)
Memory usage (with (ov|ov)): 154.935 MB
(ov|vv) shape: (7, 39, 39, 39)
(vv|vv) shape: (39, 39, 39, 39)
Memory usage (with (ov|ov), (ov|vv), (vv|vv)): 189.825 MB


`ebcc` has support for dumping and loading the state of a calculation to disk, in the format of HDF5 via the `h5py` library. Note that the contents of the DIIS object are not written via this method, so the total number of iterations required may be slightly higher in some cases.

In [5]:
import os
from ebcc import REBCC

ccsd = REBCC(mf, log=log)
ccsd.options.max_iter = 8
ccsd.kernel()

ccsd.write("restart.h5")

ccsd = REBCC.read("restart.h5", log=log)
ccsd.max_iter = 30
ccsd.kernel()

os.remove("restart.h5")


RCCSD
Options:
 > e_tol:  1e-08
 > t_tol:  1e-08
 > max_iter:  200
 > diis_space:  9
 > diis_min_space:  1
 > damping:  0.0
Ansatz: CCSD
Space: (7o, 39v)
Solving for excitation amplitudes.
Iter   Energy (corr.)      Energy (tot.)     Δ(Energy)      Δ(Ampl.)
   0    -0.2946967101    -109.2305889312
   1    -0.2929471958    -109.2288394169     1.750e-03     1.223e-02
   2    -0.3012030037    -109.2370952249     8.256e-03     3.900e-03
   3    -0.3020888054    -109.2379810265     8.858e-04     2.526e-03
   4    -0.3029697821    -109.2388620032     8.810e-04     8.621e-04
   5    -0.3029237180    -109.2388159391     4.606e-05     2.780e-04
   6    -0.3029133080    -109.2388055291     1.041e-05     1.210e-05
   7    -0.3029173594    -109.2388095805     4.051e-06     8.501e-06
   8    -0.3029174897    -109.2388097108     1.303e-07     7.707e-07
Failed to converge.
E(corr) = -0.3029174897
E(tot)  = -109.2388097108

RCCSD
Options:
 > e_tol:  1e-08
 > t_tol:  1e-08
 > max_iter:  8
 > diis_spac